In [1]:
import numpy as np
import pandas as pd

rs = np.random.RandomState(np.random.MT19937(np.random.SeedSequence(42)))

# Input
INTERNAL_RADIUS = 5.0
EXTERNAL_RADIUS = 6.0
M_1_RADIUS = 5 # 5
N_DEGREES = 4 # 4
ISOTERMA = 500
TEMP_INT = np.array([1500 for i in range(N_DEGREES)])
TEMP_EXT = np.random.normal(loc=150, scale=10, size=N_DEGREES)
RADIUSES = np.linspace(INTERNAL_RADIUS, EXTERNAL_RADIUS, M_1_RADIUS+1)
DEGREES = np.linspace(0.0, 2*np.pi, N_DEGREES)

delta_r = (EXTERNAL_RADIUS - INTERNAL_RADIUS)/(M_1_RADIUS)
delta_g = 2 * np.pi / N_DEGREES
size = (M_1_RADIUS) * (N_DEGREES)

5 6 5 4 500 1
1500 1500 1500 1500 160.8435309947052 162.42036793522266 159.59264334173378 144.6919884216488 

In [2]:
" ".join(list(map(str, TEMP_EXT)))

'142.65079644523905 145.27451252518858 164.55933389590336 161.25887207527623'

In [3]:
RADIUSES

array([5. , 5.2, 5.4, 5.6, 5.8, 6. ])

In [4]:
delta_r

0.2

In [5]:
print(INTERNAL_RADIUS, EXTERNAL_RADIUS, M_1_RADIUS, N_DEGREES, ISOTERMA, 1)
print(" ".join(list(map(str, TEMP_INT))), " ".join(list(map(str, TEMP_EXT))))

5.0 6.0 5 4 500 1
1500 1500 1500 1500 142.65079644523905 145.27451252518858 164.55933389590336 161.25887207527623


In [6]:
delta_r

0.2

In [7]:
A = np.zeros((size, size))

In [8]:
A.shape

(20, 20)

$$ A \in \mathcal{R}^{(m+1 * n) \times (m+1 * n)} $$

La ecuacion que en el estacionario dice que

\begin{equation} 
   \frac{\partial^2 T}{\partial r^2} (r, \theta) + \frac{\partial T}{r \partial r}(r, \theta) + \frac{\partial^2 T}{r^2 \partial \theta^2}(r, \theta) = 0
\end{equation}

Usando diferencias finitas tenemos que

\begin{equation} 
   \frac{t_{j-1,k} - 2 t_{j,k} + t_{j+1,k}}{(\Delta r)^2} + \frac{t_{j,k} - t_{j-1,k}}{r \Delta r} + \frac{t_{j,k-1} - 2 t_{j,k} + t_{j,k+1}}{r^2(\Delta \theta)^2} = 0
\end{equation}

Distribuyendo para obtener los coeficientes de cada variable obtenemos

1. $t_{j-1,k} = (\Delta r)^{-2} - (r \Delta r)^{-1}$
2. $t_{j , k} = (r \Delta r)^{-1} - 2(\Delta r)^{-2} - 2(r^2(\Delta\theta)^2)^{-1}$
3. $t_{j+1,k} = (\Delta r)^{-2}$
4. $t_{j,k-1} = (r^2 (\Delta\theta)^2)^{-1}$
5. $t_{j,k+1} = (r^2 (\Delta\theta)^2)^{-1}$

En total hay $(m+1) n$ puntos donde deberemos calcular la temperatura

Entonces, tenemos el vector $x=(t_{0,0},\dots,t_{0,n},t_{1,0},\dots,t_{m+1,0},\dots,t_{m+1,n})$. Si definimos la matriz $A$ de forma tal que al hacer $Ax=b$, resolvemos el sistema. Los $t_{0,k}$ y $t_{m+1,k}$ son conocidos, con $k=1,\dots,n$


Como $r, \theta, \Delta r, \Delta \theta$ son datos, los coeficientes son todos numericos ($r$ es el valor del radio en cada punto y $\theta$ el angulo, pero no son incognitas, dependen del $j,k$)

In [9]:
for i in range(1, M_1_RADIUS-1):
    for j in range(N_DEGREES):
        r = RADIUSES[i]
        # print(f"{i}:{r} - {j}:{DEGREES[j]}")
        v = np.zeros((M_1_RADIUS,N_DEGREES))
        
        v[i-1][j] = (delta_r) ** -2 - (r * delta_r) ** -1
        v[i][j] = (r * delta_r) ** -1 - 2 * (delta_r)**-2 - 2 * (r**2 * delta_g**2)**-1
        v[i+1][j] = delta_r ** -2
        tmp = (r**2 * delta_g**2) ** -1
        
        v[i][(j-1) % N_DEGREES] = tmp
        v[i][(j+1) % N_DEGREES] = tmp

        A[i*N_DEGREES+j] = v.reshape(size)

In [10]:
A.shape

(20, 20)

Revisar esto porque nunc usamos las temperaturas exteriores

In [11]:
first = np.array([-A[N_DEGREES][0] * 1500 for i in range(N_DEGREES)])
end = np.array([-delta_r**-2 * TEMP_EXT[i] for i in range(N_DEGREES)])
b = np.concatenate([first, np.zeros(A.shape[0] - 4 * N_DEGREES), end])

In [12]:
(100 - 1) * (32 - 2)

2970

In [13]:
b.shape

(12,)

In [14]:
print(b)

[-36057.69230769 -36057.69230769 -36057.69230769 -36057.69230769
      0.              0.              0.              0.
  -3566.26991113  -3631.86281313  -4113.9833474   -4031.47180188]


In [15]:
b.shape

(12,)

In [16]:
total = len(A)
App = pd.DataFrame(A).loc[N_DEGREES:total-N_DEGREES-1, N_DEGREES:total-N_DEGREES-1].to_numpy()

In [22]:
for i in range(len(App)):
    print(" ".join(list(map(lambda x: f"{str(x)}", App[i]))))

-49.06843822001252 0.014988340775493752 0.0 0.014988340775493752 24.999999999999996 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.014988340775493752 -49.06843822001252 0.014988340775493752 0.0 0.0 24.999999999999996 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.014988340775493752 -49.06843822001252 0.014988340775493752 0.0 0.0 24.999999999999996 0.0 0.0 0.0 0.0 0.0
0.014988340775493752 0.0 0.014988340775493752 -49.06843822001252 0.0 0.0 0.0 24.999999999999996 0.0 0.0 0.0 0.0
24.07407407407407 0.0 0.0 0.0 -49.10187138097183 0.013898653448880352 0.0 0.013898653448880352 24.999999999999996 0.0 0.0 0.0
0.0 24.07407407407407 0.0 0.0 0.013898653448880352 -49.10187138097183 0.013898653448880352 0.0 0.0 24.999999999999996 0.0 0.0
0.0 0.0 24.07407407407407 0.0 0.0 0.013898653448880352 -49.10187138097183 0.013898653448880352 0.0 0.0 24.999999999999996 0.0
0.0 0.0 0.0 24.07407407407407 0.013898653448880352 0.0 0.013898653448880352 -49.10187138097183 0.0 0.0 0.0 24.999999999999996
0.0 0.0 0.0 0.0 24.107142857142854 0.0 0.0 0.0 -

In [57]:
App.shape

(12, 12)

In [58]:
App[1]

array([ 1.49883408e-02, -4.90684382e+01,  1.49883408e-02,  0.00000000e+00,
        0.00000000e+00,  2.50000000e+01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [59]:
x = np.linalg.solve(App, b)

In [60]:
x[:N_DEGREES].__len__()

4

In [61]:
x[4]

794.0228972161976

In [62]:
x

array([1140.0916716 , 1143.74134792, 1147.30531677, 1141.5693229 ,
        794.02289722,  801.18500307,  808.1813893 ,  796.92188806,
        460.76589142,  471.31602239,  481.62775659,  465.03452503])

In [63]:
RADIUSES

array([5. , 5.2, 5.4, 5.6, 5.8, 6. ])

In [71]:
x

array([1140.0916716 , 1143.74134792, 1147.30531677, 1141.5693229 ,
        794.02289722,  801.18500307,  808.1813893 ,  796.92188806,
        460.76589142,  471.31602239,  481.62775659,  465.03452503])

In [73]:
x[(1+1) * 4 + 0]

460.76589141573527

In [69]:
for i in range(M_1_RADIUS):
    print(i, INTERNAL_RADIUS + delta_r * i)

0 5.0
1 5.2
2 5.4
3 5.6
4 5.8


In [23]:
xx = x.reshape((int(x.shape[0] / N_DEGREES), N_DEGREES))

In [27]:
import scipy.linalg

_,_, u = scipy.linalg.lu(App)
# print(u)